In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw10.ipynb")

<div class="alert alert-success" markdown="1">

#### Homework 10

# Cross-Validation, Regularization, Gradient Descent, and Logistic Regression

### EECS 398-003: Practical Data Science, Fall 2024

#### Due Monday, December 2nd at 11:59PM (note the later deadline!)
    
</div>

## Instructions

Welcome to Homework 10! In this homework, you'll:

- implement cross-validation to choose model hyperparameters,
- understand why ridge regression works the way it does, down to the linear algebraic details,
- build sophisticated `sklearn` modeling Pipelines,
- trace through the steps of gradient descent and see how convexity plays a role,
- and start familiarizing yourself with the logistic regression model for classification.

This homework touches on ideas from Lectures 19 through 24; the content in the homework roughly appears in the order in which we covered it in class. See the [Readings section of the Resources tab on the course website](https://practicaldsc.org/resources/#readings) for supplemental resources.

You are given **eight** slip days throughout the semester to extend deadlines. See the [Syllabus](https://practicaldsc.org/syllabus) for more details. With the exception of using slip days, late work will not be accepted unless you have made special arrangements with your instructor.

To access this notebook, you'll need to clone our [public GitHub repository](https://github.com/practicaldsc/fa24/). The [⚙️ Environment Setup](https://practicaldsc.org/env-setup) page on the course website walks you through the necessary steps.
<div class="alert alert-warning" markdown="1">

<div class="alert alert-warning">
This homework features a mix of autograded programming questions and manually-graded questions.
    
- Questions 2, 4, 5, and 6 are **manually graded**, like in Homework 9, and say **[Written ✏️]** in the title. For these questions, **do not write your answers in this notebook**! Instead, like in Homework 9, write **all** of your answers to the written questions in this homework in a separate PDF. You can create this PDF either digitally, using your tablet or using [Overleaf + LaTeX](https://overleaf.com) (or some other sort of digital document), or by writing your answers on a piece of paper and scanning them in. Submit this separate PDF to the **Homework 10 (Questions 2, 4, 5, and 6; written problems)** assignment on Gradescope, and **make sure to correctly select the pages associated with each question**!

- Questions 1 and 3 are **fully autograded**, and say **[Autograded 💻]** in the title. For these questions, all you need to is write your code in this notebook, run the local `grader.check` tests, and submit to the **Homework 10 (Questions 1 and 3; autograded problems)** assignment on Gradescope to have your code graded by the hidden autograder.

- Question 3.4 is an **extra-credit prediction competition**. If you choose to participate, you'll need to upload your test set predictions in Question 3.4 to the separate **Homework 10, Question 3.4 Leaderboard (Optional)!** assignment.

Your Homework 10 submission time will be the **later** of your three individual submissions.
</div>
</div>

**Make sure to show your work for all written questions! Answers without work shown may not receive full credit.**

This homework is worth a total of **80 points**, 56 of which are manually graded and 24 of which come from the autograder. This is not including the potential extra credit provided by Question 3.4, which only 5 students in the class can receive (see Question 3.4 for more details). The number of points each question is worth is listed at the start of each question. **All questions in the assignment are independent, so feel free to move around if you get stuck**, but keep in mind that you'll need to submit this homework twice – one submission for your written problems, and one submission for your autograded problems. Tip: if you're using Jupyter Lab, you can see a Table of Contents for the notebook by going to View > Table of Contents.

To get started, run the cell below, plus the cell at the top of the notebook that imports and initializes `otter`. 

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

# Preferred styles
pio.templates["pds"] = go.layout.Template(
    layout=dict(
        margin=dict(l=30, r=30, t=30, b=30),
        autosize=True,
        width=600,
        height=400,
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True),
        title=dict(x=0.5, xanchor="center"),
    )
)
pio.templates.default = "simple_white+pds"

# Use plotly as default plotting engine
pd.options.plotting.backend = "plotly"

## Question 1: $k$-Nearest Neighbors Returns! 🏡🏠

---

In Homework 9, you implemented $k$-Nearest Neighbors Regression. For a refresher of how the method words, review the writeup to Question 3.3 on [Homework 9](https://github.com/practicaldsc/fa24/blob/main/homeworks/hw09/hw09.ipynb). (In Lecture 23, we also learned about the $k$-Nearest Neighbors classifier; the classifier and regressor work similarly, but our exploration here is about the **regressor**.)

In $k$-NN Regression, $k$ was a **hyperparameter** – you got to choose it before the model was fit to the data. In Question 3.4, we had you estimate, intuitively, a value of $k$ that would create a regressor that generalized well to unseen data. In this question, we'll have you use a more principled approach – cross-validation. And, you'll have to implement the cross-validation yourself!

Let's start by loading in the same `homeruns` dataset from Homework 9, Question 3.

In [ ]:
homeruns = pd.read_csv('data/homeruns.csv')
homeruns.head()

In [ ]:
homeruns.plot(kind='scatter', x='Year', y='Homeruns')

We'll continue trying to predict `'Homeruns'` as a function of `'Year'`. Last time, we had you implement $k$-Nearest Neighbors Regression by hand. Since you know how to do that already, here, we'll use `sklearn`'s implementation. `KNeighborsRegressor` is imported for you below, along with another useful function.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

### Question 1.1 [Autograded 💻] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">1 Point</div>

Assign `X_train`, `X_test`, `y_train`, and `y_test` to the result of performing a train-test split on `homeruns`. Use the default train-test split size, and set `random_state=98`.

In [ ]:
X_train, X_test, y_train, y_test = ...

In [ ]:
grader.check("q01_01")

### Question 1.2 [Autograded 💻] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">5 Points</div>

Eventually, we'll want to use cross-validation to identify the value of $k$ – that is, the number of neighbors – that specifies a model that best generalizes to unseen data. But first, we need a function that can perform cross-validation for us.

While `GridSearchCV` and `cross_val_score` exist, **you cannot use them in this question** – instead, the goal here is to implement cross-validation yourself to help really understand how it works.

<div class="alert alert-warning">
    
Note that the terminology is a little confusing, since we're using $k$-fold cross-validation to choose a $k$ for $k$-Nearest Neighbors Regression.

<b>In this question, $k$ will always refer to the number of neighbors to use in $k$-Nearest Neighbors Regression.</b> We'll use other terminology to refer to the number of folds in cross-validation.
    
</div>

Complete the implementation of the function `cross_validate_model`, which takes in:
- `model`, an **un-fit** instance of an `sklearn` estimator object, like `LinearRegression()` or `KNeighborsRegressor(10)`,
- `X_train`, a 2D array/DataFrame with $x$-values being used to train a model.
- `y_train`, a 1D array/Series with $y$-values being used to train a model, with the same number of rows as `X_train`.
- `cv`, a number of folds to use for cross-validation (normally, we call this the $k$ in $k$-fold cross-validation).

`cross_validate_model` should implement `cv`-fold cross-validation, as described [**here in Lecture 20**](https://practicaldsc.org/resources/lectures/lec20/lec20-filled.html#Illustrating-$k$-fold-cross-validation). Specifically, it should:
- Divide `X_train` and `y_train` into `cv` disjoint folds of equal size.
    - `cross_validate_model` **should not** shuffle before creating these folds – instead, it should divide the data as-is into the folds.
    - For example, if `X_train` and `y_train` have 30 rows, and `cv = 3`, fold 0 should be rows 0-9, fold 1 should be rows 10-19, and fold 2 should be rows 20-29.
- Train `model` `cv` times, such that:
    - Each fold is used for validation once and for training `cv - 1` times.
    - Each time `model` is trained, compute its **validation mean squared error** on the validation fold.
- Return a **DataFrame** with `cv` rows and 2 columns, `'training_mse'` and `'validation_mse'`.
    - There should be one row per fold; the index of the returned DataFrame should be `'Fold 0'`, `'Fold 1'`, and so on.
    - If `out` is the returned DataFrame, then for example, `out.loc['Fold 4', 'training_mse']` should be the training mean squared error when fold 4 was used for validation (and the other `cv - 1` folds were used for training) and `out.loc['Fold 4', 'validation_mse']` should be the validation mean squared error when fold 4 was used for validation.
    - The example above assumes that `cv >= 5`; note that in general, the only restriction on `cv` is that `cv >= 2`.
    

Example behavior is given below.

```python
>>> out = cross_validate_model(KNeighborsRegressor(2), X_train, y_train, 10)
>>> out
```
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>training_mse</th>
      <th>validation_mse</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Fold 0</th>
      <td>49681.725309</td>
      <td>192754.750000</td>
    </tr>
    <tr>
      <th>Fold 1</th>
      <td>66545.209877</td>
      <td>100492.833333</td>
    </tr>
    <tr>
      <th>Fold 2</th>
      <td>57809.941358</td>
      <td>121249.750000</td>
    </tr>
    <tr>
      <th>Fold 3</th>
      <td>61673.419753</td>
      <td>186130.000000</td>
    </tr>
    <tr>
      <th>Fold 4</th>
      <td>64469.604938</td>
      <td>58628.527778</td>
    </tr>
    <tr>
      <th>Fold 5</th>
      <td>44152.598765</td>
      <td>284344.472222</td>
    </tr>
    <tr>
      <th>Fold 6</th>
      <td>56141.070988</td>
      <td>55286.444444</td>
    </tr>
    <tr>
      <th>Fold 7</th>
      <td>60590.222222</td>
      <td>379158.166667</td>
    </tr>
    <tr>
      <th>Fold 8</th>
      <td>64147.987654</td>
      <td>69341.138889</td>
    </tr>
    <tr>
      <th>Fold 9</th>
      <td>70259.151235</td>
      <td>89080.527778</td>
    </tr>
  </tbody>
</table>

For more context on the example above:
- `X_train` and `y_train` are divided into 10 folds. `X_train` and `y_train` each have 90 rows, so each fold has 9 points.
- When fold 0 is used for validation:
    - Fold 0 corresponds to rows 0-8 of `X_train` and `y_train`.
    - The rows used for training, then, are folds 1-9, i.e. rows 9-89 of `X_train` and `y_train`.
    - A `KNeighborsRegressor(2)` instance is fit on rows 9-89 of `X_train` and `y_train`.
    - The mean squared error of that model instance, when evaluated on rows 9-89, is `49681.725309`, so `out.loc['Fold 0', 'training_mse']` is `49681.725309`.
    - The mean squared error of that model instance, when evaluated on rows 0-8, is `192754.75`, so `out.loc['Fold 0', 'validation_mse']` is `192754.75`.
- When fold 1 is used for validation:
    - Fold 1 corresponds to rows 9-17 of `X_train` and `y_train`.
    - The rows used for training, then, are folds 0, 2, 3, 4, ..., 9, i.e. rows 0-8 and rows 18-89 of `X_train` and `y_train`. **A big part of the question is determining, programmatically, which rows are to be used for training!**
    - A `KNeighborsRegressor(2)` instance is fit on rows 0-8 and 18-89 of `X_train` and `y_train`.
    - The mean squared error of that model instance, when evaluated on rows 0-8 and 18-89, is `66545.209877`, so `out.loc['Fold 1', 'training_mse']` is `66545.209877`.
    - The mean squared error of that model instance, when evaluated on rows 9-17, is `100492.83333333`, so `out.loc['Fold 1', 'validation_mse']` is `100492.83333333`.
- And so on!
    

Some guidance:
- Assume that the number of rows in `X_train` is divisible by `cv`, i.e. assume all folds are of the same size. Furthermore, assume that `cv >= 2`.
- Assume that `X_train` and `y_train` have the same number of rows, but **don't** assume that they have the same index! The `X_train` and `y_train` you produced in Question 1.1 do have the same index, but we should make your `cross_validate_model` more general-purpose.
- Remember that `model` could be any un-fit `sklearn` estimator instance, not just `KNeighborsRegressor(2)`.
- Remember that **you must implement cross-validation from scratch here – you cannot use any pre-built implementation of it**. The animation in Lecture 20 will be helpful.
    - If it helps you in understanding what the goal is, note that `cross_validate_model` does something very similar to `cross_val_score` from lecture – but again, you can't use it (we're checking!).
    - **You can't use `sklearn`'s `mean_squared_error` either – please implement it yourself, we'll be checking!**
- You can use a `for`-loop – our solution had two (more specifically, one loop and one list comprehension).

In [ ]:
def cross_validate_model(model, X_train, y_train, cv):
    # Remember: Do **not** shuffle X_train or y_train here;
    # train_test_split already did the shuffling for us!
    ...
    
# Feel free to change this input to make sure your function works correctly.
cross_validate_model(KNeighborsRegressor(2), X_train, y_train, 10)

In [ ]:
grader.check("q01_02")

### Question 1.3 [Autograded 💻] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">5 Points</div>

Now, let's use your implementation of `cross_validate_model` to find the value of $k$ – that is, the number of neighbors – that best generalizes to unseen data.

Complete the implementation of the function `plot_validation_error_vs_k`, which takes in:
- `k_max`, a positive integer.
- `X_train`, `y_train`, and `cv`, all of which are defined the same way as when implementing `cross_validate_model`.

`plot_validation_error_vs_k` should return a `plotly` Figure object like the one below:

<br>

<center><img src="imgs/knn.png" width=550></center>

There are several steps involved here.
- `plot_validation_error_vs_k` should call `cross_validate_model` `k_max` times.
    - Once, `cross_validate_model` should be called with `model = KNeighborsRegressor(1)`.
    - Then, `cross_validate_model` should be called with `model = KNeighborsRegressor(2)`.
    - and so on, until `model = KNeighborsRegressor(k_max)`.
    - Each time `cross_validate_model` is called, `X_train`, `y_train`, and `cv` should all be passed as-is without modification.
- After calling `cross_validate_model` for a particular value of $k$, you should compute the **average** training MSE and **average** validation MSE for that $k$ and store it somewhere.
- Then, create a DataFrame (likely, one that has $k$ rows and 2 columns) with the average training and validation MSE for each value of $k$, and create a `plotly` line chart with the values in that DataFrame.
- Some properties that must be true of your `plotly` Figure:
    - It must have exactly two lines.
    - The two lines should have different names in the legend; one should have `'training'` somewhere in the name (in any case), and the other should have `'validation'`, and these names should correspond to the types of errors the lines are depicting.
    - The $x$-axis and $y$-axis titles must be exactly the same as ours (including capitalization).
    - The $x$-axis ticks should say `'k = 1'`, `'k = 2'`, and so on, as well. It doesn't matter if the tick labels are rotated on an angle or not (this is determined automatically by `plotly`, depending on what you set `k_max` to, and is not super relevant).

In [ ]:
def plot_validation_error_vs_k(k_max, X_train, y_train, cv):
    ...
    
# Feel free to change this input to make sure your function works correctly.
plot_validation_error_vs_k(20, X_train, y_train, 5)

In [ ]:
grader.check("q01_03")

Now that you've completed `plot_validation_error_vs_k`, let's take a look at the results one more time:

In [ ]:
plot_validation_error_vs_k(20, X_train, y_train, 5)

You reflected on the behavior of $k$ in $k$-Nearest Neighbors Regression models last week, and gave an intuitive choice for a "good" value of $k$.

Now, it's unambiguously clear what the "best" choice of $k$ is: $k = 4$. But, any choice in the range of $k = 3$ to $k = 11$ seems to produce roughly the same average validation mean squared error. Note that this plot looks a little different than the plots of training/validation error vs. model complexity that we saw in lecture because **here, as Number of Neighbors ($k$) increases, model complexity decreases**. $k = 1$ is the most overfit model, since it simply memorizes the $(x_i, y_i)$ pairs in the dataset. In our first lecture example, as our hyperparameter (there, polynomial degree) increased, model complexity increased, too.

Nice job! You've manually implemented every single calculation that produced the results above. In the last homework, you implemented the regressor yourself, and in this homework you cross-validated it yourself.

## Question 2: Ruffles Have Ridges ⛰️

---

As we saw in [Lecture 21](https://practicaldsc.org/resources/lectures/lec21/lec21-filled.html#ridge-regression), **ridge regression** is the problem of finding the vector $\vec{w}$ that minimizes the following **objective function**:

$$R_\text{ridge}(\vec{w}) = \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert_2^2 + \lambda \sum_{j = 1}^d w_j^2$$

Once we find that vector, we can make predictions using $\vec{h} = X \vec{w}_\text{ridge}^*$, where $X$ is a design matrix with information about the individuals we want to make predictions for.

The vector that minimizes $R_\text{ridge}(\vec{w})$ is:

$$\vec{w}_\text{ridge}^* = (X^TX + n \lambda I)^{-1} X^T \vec{y}$$

$\vec{w}_\text{ridge}^*$ is unique, whether or not $X$ is full rank. 

This is different from $\vec{w}_\text{OLS}^* = (X^TX)^{-1}X^T \vec{y}$, which is only uniquely defined when $X^TX$ is invertible; otherwise, all of the infinitely many solutions to the normal equations, $X^TX \vec{w}_\text{OLS}^* = X^T \vec{y}$, minimize mean squared error. Remember, "OLS" refers to "ordinary least squares", the process of minimizing mean squared error, $R_\text{sq}(\vec{w}) = \frac{1}{n} \lVert \vec y - X \vec w \rVert_2^2$, with no regularization.

Some lingering questions:
- When there are infinitely many solutions to the normal equations, which solution(s) does Python return?
- Why is it called ridge regression?
- Why is $\vec{w}_\text{ridge}^*$ always uniquely-defined, i.e. why does the ridge regression objective function always have a unique solution?

Let's explore! There's a lot of code in this question, but it's all already written for you. Your job is to work through our exploration, and then answer the questions at the end. **But don't just skip this exploration** – it's important in building your conceptual understanding of ridge regression (and will likely be helpful in studying for the Final Exam!).

Below, we load in the dataset containing the weights and heights of 25,000 18 year olds we used in Lecture 18 to demonstrate multicollinearity.

In [ ]:
people = pd.read_csv('data/heights-weights.csv')
people.head()

Note that there are two height columns, `'Height (inches)'` and `'Height (cm)'`. Remember that 1 inch is equal to 2.54 centimeters.

Throughout this question, we'll aim to predict `'Weight (Pounds)'` using the other two features. Let's start by plotting `'Weight (Pounds)'` vs. `'Height (Inches)'`:

In [ ]:
px.scatter(people, x='Height (Inches)', y='Weight (Pounds)', title='Weight vs. Height for 25,000 18 Year Olds')

And `'Weight (Pounds)'` vs. `'Height (Inches)'` and `'Height (cm)'`:

In [ ]:
px.scatter_3d(people, x='Height (Inches)', y='Height (cm)', z='Weight (Pounds)', title='Weight vs. Height for 25,000 18 Year Olds')

Drag the plot above around. You should notice that the points form a flat "patty" that resembles the 2D plot from above, not a cloud ☁️
like in other 3D scatter plots we've seen in class. This is because `'Height (cm)'` and `'Height (Inches)'` are the same values, just in different units. For a particular `'Height (Inches)'` value, there is only one possible `'Height (cm)'` value – specifically, 2.54 times the `'Height (Inches)'` value – and so all points in the plot sit on the flat plane:

$$\text{Height (cm)} = 2.54 \cdot \text{Height (Inches)}$$

We'll start by fitting a multiple linear regression model to predict `'Weight (Pounds)'` from `'Height (Inches)'` and `'Height (cm)'`, without regularization. Our model is of the form:

$$\text{predicted Weight (Pounds)} = w_0 + w_1 \cdot \text{Height (Inches)} + w_2 \cdot \text{Height (cm)}$$

First, a train-test split:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(people[['Height (Inches)', 'Height (cm)']], 
                                                    people['Weight (Pounds)'],
                                                    random_state=98)

The design matrix for our training data, `X_train_design`, is defined below:

In [ ]:
X_train_design = X_train.copy()
X_train_design['1'] = 1
X_train_design = X_train_design[['1', 'Height (Inches)', 'Height (cm)']]
X_train_design

We know that our design matrix isn't full rank. Python knows this, too:

In [ ]:
np.linalg.matrix_rank(X_train_design)

Note that $X$ and $X^TX$ have the same rank:

In [ ]:
np.linalg.matrix_rank(X_train_design.T @ X_train_design)

So, $X^TX$ is not invertible, and there are infinitely many solutions to the normal equations, 

$$X^TX \vec{w}_\text{OLS}^* = X^T \vec{y}$$

No line of code can return an infinitely long output (at least, not in finite time), so there's no way to see **all** of the infinitely many possible solutions in code. Instead, to find the relationship between the infinitely many possible $\vec{w}^*_\text{OLS}$ values, we'd need to think about the relationship between our multicollinear features, like we did in [Lecture 18](https://practicaldsc.org/resources/lectures/lec18/lec18-filled.html#Redundant-features).

It turns out that there are several different ways we can _try_ and solve the normal equations in Python – and many of them give different results! Let's enumerate some possibilities.

1️⃣ We can try and use `np.linalg.inv` and try and evaluate $\vec{w}_\text{OLS}^* = (X^TX)^{-1} X^T \vec{y}$ directly. It's not clear why this should work, since $X^TX$ is **not** invertible. Nevertheless, Python gives us _some_ result!

In [ ]:
w_inv = np.linalg.inv(X_train_design.T @ X_train_design) @ X_train_design.T @ y_train
w_inv = w_inv.to_numpy()
w_inv

2️⃣ We can use the same approach as above, but instead use `np.linalg.pinv`, which computes the **pseudoinverse** of its argument. The pseudoinverse is the generalization of the matrix inverse to non-invertible matrices; read more [here](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse).

In [ ]:
w_pinv = np.linalg.pinv(X_train_design.T @ X_train_design) @ X_train_design.T @ y_train
w_pinv = w_pinv.to_numpy()
w_pinv

3️⃣ We can use `np.linalg.solve` to "solve" the normal equations, $X^TX \vec{w}_\text{OLS}^* = X^T \vec{y}$, without needing to invert.

In [ ]:
w_solve = np.linalg.solve(X_train_design.T @ X_train_design, X_train_design.T @ y_train)
w_solve

4️⃣ We can also use `np.linalg.lstsq`. `np.linalg.lstsq(A, B)` returns the vector $\vec{w}$ that minimizes $\frac{1}{n} \lVert B - A \vec{w} \rVert_2^2$. For us, `A = X_train_design` and `B = y_train`.

In [ ]:
w_lstsq = np.linalg.lstsq(X_train_design, y_train)[0]
w_lstsq

5️⃣ Finally, we can just use `sklearn`.

In [ ]:
from sklearn.linear_model import LinearRegression

# Our design matrix already has a column of 1s, so we don't need to tell sklearn to fit an intercept.
model = LinearRegression(fit_intercept=False) 
model.fit(X_train_design, y_train)

w_sklearn = model.coef_
w_sklearn

Hmm... all five methods were in theory solving the same problem, and so should have given us the same optimal parameter vector, $\vec{w}_\text{OLS}^*$, but the first four were slightly different. The last two were identical; upon further investigation, [`sklearn`'s documentation](https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LinearRegression.html) shows us that LinearRegression's `fit` method just calls `np.linalg.lstsq` under the hood!

While many of the $\vec{w}_\text{OLS}^*$ vectors look different, many (but not all!) end up making the same predictions and have the same mean squared error. Let's take a look:

In [ ]:
def display_w(w, name):
    from IPython.display import Markdown
    display(Markdown(f'#### Using {name}'))
    display(w)
    display(Markdown('First three rows of hypothesis vector:'))
    display((X_train_design @ w).head(3))
    display(Markdown(f'Mean squared error: {np.mean((y_train - X_train_design @ w) ** 2)}'))

display_w(w_inv, 'inverse')
display_w(w_pinv, 'pseudoinverse')
display_w(w_solve, 'solve')
display_w(w_lstsq, 'lstsq')
display_w(w_sklearn, 'sklearn')

It seems that the last four $\vec{w}_\text{OLS}^*$ vectors made the same predictions and had the same minimal mean squared error of $\approx 101.31$, while the optimal $\vec{w}_\text{OLS}^*$ found using `np.linalg.inv` **wasn't** actually optimal, since it didn't have the minimal mean squared error! (There were minor differences between the last four, but these differences are attributed to the fact that the techniques use different numerical methods to solve for the necessary vectors with different stopping criteria, and due to the general imprecision of floating-point arithmetic.)

This tells us that `np.linalg.inv` does _something_ when the input matrix is not invertible, and the result isn't reliable, at least not in the context of solving the normal equations.

But generally, it seems that when the design matrix isn't full rank, we still can minimize mean squared error, it's just unclear which "optimal" parameter vector $\vec{w}_\text{OLS}^*$ we'll end up with, since there are infinitely many choices that minimize MSE. This is an issue if we care about interpreting the coefficients of our model.

**So, how does ridge regression help with this problem?**

To illustrate, let's look at a graph of $R_\text{ridge}(\vec{w}) = \frac{1}{n} \lVert \vec y - X \vec w \rVert_2^2 + \lambda \sum_{j = 1}^d w_j^2$, for different values of $\lambda$. The graph you're about to see is also known as a _loss surface_.

Remember, our model is of the form:

$$\text{predicted Weight (Pounds)} = w_0 + w_1 \cdot \text{Height (Inches)} + w_2 \cdot \text{Height (cm)}$$

There are three axes in the graph that appears:

- One for different values of $w_1$.
- One for different values of $w_2$.
- One for the value of $R_\text{ridge}(-82.77, w_1, w_2)$. Since we have three parameters, we'd need four axes to truly visualize $R_\text{ridge}(\vec{w})$, so we've fixed a particular value of $w_0$. (The interesting part of the graph doesn't involve the intercept, anyways, since the multicollinearity is between `'Height (Inches)'` and `'Height (cm)'`.)

Run the cell below and interact with the slider that appears!

In [ ]:
def show_loss_surface(reg_lambda):
    def mse_for_weight_model(w):
        w = np.array([-82.77, w[0], w[1]])
        return np.mean((y_train - X_train_design @ w) ** 2) + reg_lambda * (w[0] ** 2 + w[1] ** 2)

    if reg_lambda > 0:
        identity = np.eye(X_train_design.shape[1])
        identity[0, 0] = 0
        w_star = np.linalg.solve(X_train_design.T @ X_train_design + X_train_design.shape[0] * reg_lambda * identity,
                                 X_train_design.T @ y_train)

    num_points = 25
    uvalues = np.linspace(-1.5, 2, num_points)
    vvalues = np.linspace(-2, 5, num_points)
    (u,v) = np.meshgrid(uvalues, vvalues)
    ws = np.vstack((u.flatten(), v.flatten()))
    MSE = np.array([mse_for_weight_model(w) for w in ws.T])
    loss_surface = go.Surface(x=u, y=v, z=np.reshape(MSE, u.shape), showscale=False)

    fig = go.Figure(data=[loss_surface])
    # fig.add_trace(opt_point)
    fig.update_layout(title=("Mean Squared Error" if reg_lambda == 0 else f"Ridge Regression with Lambda = {reg_lambda}"), scene = dict(
        xaxis_title = "w1: Height (Inches)",
        yaxis_title = "w2: Height (cm)"),
                     width=700, height=500, showlegend=False)
    fig.show()

from ipywidgets import interact
interact(show_loss_surface, reg_lambda=(0, 100000, 5000));

Some things to note:
- When you drag the `reg_lambda` slider to 0, and see the title "Mean Squared Error", you should notice a long "ridge" at the bottom of the loss surface! All values of $w_1$ and $w_2$ that fall on that ridge minimize mean squared error. **This ridge is problematic!**
- As you increase `reg_lambda`, the surface looks more and more like a bowl curved upwards, and less "ridgy". **Ridge regression removes the ridge!**
- As you increase `reg_lambda`, look at the $z$-axis of the resulting plot.

So, there, we have our answer! Ridge regression removes the "ridge" of infinitely many solutions when multicollinearity is present.

What we _haven't_ really answered is **why?** Why does the ridge regression objective function always have a unique minimizer, $\vec{w}_\text{ridge}^*$? That's what we'll work through now.

<br>

**Throughout the remainder of this question question, we'll call the regularization penalty $\lambda_\text{ridge}$, not just $\lambda$, to avoid confusion with other notation we're about to introduce.**

We've just taken for granted the fact that the minimizer of the ridge regression objective function,

$$R_\text{ridge}(\vec{w}) = \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert_2^2 + \lambda_\text{ridge} \sum_{j = 1}^d w_j^2$$

is:

$$\vec{w}_\text{ridge}^* = (X^TX + n \lambda_\text{ridge} I)^{-1} X^T \vec{y}$$

In Questions 2.4 through 2.6, we'll prove that this is indeed the minimizer. But for now, we'll answer the question, **why is $\vec{w}_\text{ridge}^*$ always uniquely defined?** This problem boils down to determining why $X^TX + n \lambda_\text{ridge} I$ is always invertible, since the inverse of a matrix – **if it exists** – is always unique.

To prove that $X^TX + n \lambda_\text{ridge} I$ is always invertible, we'll need to remember how eigenvalues and eigenvectors work from linear algebra. If you don't remember (or never learned), don't worry – we'll explain everything that's relevant. (And yes, this is all relevant to understanding how ridge regression works!)

$\lambda_i$ is an **eigenvalue** of square matrix $A$, corresponding to the **eigenvector** $\vec{v}_i$, if:

$$A \vec{v}_i = \lambda_i \vec{v}_i$$

In other words, $\vec{v}_i$ is an eigenvalue of $A$ if, when left-multiplied by $A$, its direction doesn't change, only its length. The amount $\vec{v}_i$'s length is scaled by is $\lambda_i$. We also say that $\lambda_i$ and $\vec{v}_i$ form an **eigenvalue-eigenvector pair of $A$**. (Note that $\vec{0}$ is never considered to be an eigenvector, since any matrix times $\vec{0}$ is always just $\vec{0}$.)

For example, if $A = \begin{bmatrix} -5 & 2 \\ -7 & 4 \end{bmatrix}$, then $\vec{v}_1 = \begin{bmatrix} 2 \\ 7 \end{bmatrix}$ is an eigenvector of $A$ corresponding to the eigenvalue $\lambda_1 = 2$, because:

$$A\vec{v}_1 = \begin{bmatrix} -5 & 2 \\ -7 & 4 \end{bmatrix} \begin{bmatrix} 2 \\ 7 \end{bmatrix} = \begin{bmatrix} -5(2) + 2(7) \\ -7(2) + 4(7) \end{bmatrix} = \begin{bmatrix} 4 \\ 14 \end{bmatrix} = 2 \begin{bmatrix} 2 \\ 7 \end{bmatrix} = 2 \vec{v}_1$$

So, when $\vec{v}_1$ is multiplied by $A$, it still points in the same direction, it's just doubled in length.

**Verify yourself that $\lambda_2 = -3$ is also an eigenvalue of $A$, corresponding to the eigenvector $\vec{v}_2 = \begin{bmatrix} 1 \\ 1\end{bmatrix}$.** Read more about eigenvalues and eigenvectors [**here**](https://math.libretexts.org/Bookshelves/Linear_Algebra/A_First_Course_in_Linear_Algebra_(Kuttler)/07%3A_Spectral_Theory/7.01%3A_Eigenvalues_and_Eigenvectors_of_a_Matrix), though we will cover everything you need to know directly in this homework.

One more thing: an $n \times n$ matrix can have at most $n$ non-zero eigenvalues. The **rank** of a square matrix is equal to the number of non-zero eigenvalues it has.

<br>

Okay, so what did we need all of that for? It's to use this fact:

<center><b>A square matrix is invertible <i>if and only if</i> none of its eigenvalues are 0.</b></center>

We will use this particular fact without proof, mostly because its proof is similar to the proof you're about to do yourself. If we can prove that none of $X^TX + n \lambda_\text{ridge} I$'s eigenvalues are 0, then we know it must be invertible, and so $\vec{w}_\text{ridge}^*$ is uniquely defined. This is especially useful if some of $X^TX$'s eigenvalues are 0, which is the case when $X$ (and $X^TX$) isn't full rank, and hence $X^TX$ isn't invertible.


It's time to get started.

<!-- BEGIN QUESTION -->

### Question 2.1 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

Prove that all of the eigenvalues of $X^TX$, where $X$ is the design matrix, are non-negative.

Some guidance:
- Start by letting $\lambda_i$ and $\vec{v}_i$ be an arbitrary eigenvalue-eigenvector pair of $X^TX$. By the definition of eigenvalues and eigenvectors, what does this mean? (Hint: can $\vec{v}_i = \vec{0}$?)
- Left-multiply both sides of the equation by $\vec{v}_i^T$. What does this give us?
- From here, use the facts that $(AB)^T = B^T A^T$ and that $\lVert \vec u \rVert^2 = \vec u \cdot \vec u$ to show that $\lambda_i \geq 0$.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 2.2 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

If $\lambda_i$ and $\vec{v}_i$ are an eigenvalue-eigenvector pair of $X^TX$, then show that $\vec{v}_i$ is **also** an eigenvector of $X^TX + n \lambda_\text{ridge} I$, with a different eigenvalue. What is the corresponding eigenvalue?

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 2.3 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Putting the results of 2.1 and 2.2 together, explain why it is **guaranteed** that $X^TX + n \lambda_\text{ridge} I$ is invertible, for any $\lambda_\text{ridge} > 0$.

<!-- END QUESTION -->

<br>

Great – so now we know _why_ the ridge regression objective function always has a unique minimizer, even when $X^TX$ itself isn't invertible.

But why is the minimizer of $R_\text{ridge}(\vec{w}) = \frac{1}{n} \lVert \vec{y} - X \vec{w} \rVert_2^2 + \lambda_\text{ridge} \sum_{j = 1}^d w_j^2$ the vector that we're claiming it is, $\vec{w}_\text{ridge}^* = (X^TX + n \lambda_\text{ridge} I)^{-1}X^T \vec{y}$? Let's show that final bit here. But first, a bit more setup.

Recall, our design matrix $X$ and observation vector $\vec{y}$ are defined as follows:

$$X = \begin{bmatrix}1 & x_1^{(1)} & x_1^{(2)} & ... & x_1^{(d)} \\ 1 & x_2^{(1)} & x_2^{(2)} & ... & x_2^{(d)} \\ \vdots & \vdots & \vdots & \vdots & \vdots \\ 1 & x_n^{(1)} & x_n^{(2)} & ... & x_n^{(d)} \end{bmatrix}_{\: n \times (d + 1)} \qquad \vec{y} = \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix}_{\:n \times 1}$$

Recall from [Lecture 16](https://practicaldsc.org/resources/lectures/lec16/lec16-filled.pdf#page=11) that the role of the intercept term in a linear model is to shift our hypothesis surface up, ensuring that an average $\vec{x}_i$ predicts an average $y_i$. If we **mean-center** each column in our design matrix, $X$, and our observation vector, $\vec{y}$, then there's no need for an intercept term in our model, since the mean of each feature will be $0$ and the mean $y$ will also be 0.

Let $\mu_j$ be the mean of feature/column $j$, and let $\bar{y}$ be the mean of $\vec y$. Then, **mean-centered** design matrix, $X_c$, and observation vector, $\vec{y}_c$, are defined as follows:

$$X_c = \begin{bmatrix}x_1^{(1)} - \mu_1 && x_1^{(2)} - \mu_2 && ... && x_1^{(d)} - \mu_d \\ x_2^{(1)} - \mu_1 && x_2^{(2)} - \mu_2 && ... && x_2^{(d)} - \mu_d \\ \vdots && \vdots && \vdots && \vdots \\ x_n^{(1)} - \mu_1 && x_n^{(2)} - \mu_2 && ... && x_n^{(d)} - \mu_d \end{bmatrix}_{\:n \times d} \qquad \vec{y} = \begin{bmatrix} y_1 - \bar{y} \\ y_2 - \bar{y} \\ \vdots \\ y_n - \bar{y} \end{bmatrix}_{\:n \times 1}$$

<div class="alert alert-warning" markdown="1">
    
**Fact**: The vector $\vec{w}_c^*$ that minimizes mean squared error using the centered design matrix and observation vector, $R_\text{sq-c}(\vec{w}) = \frac{1}{n} \lVert \vec{y}_c - X_c \vec{w} \rVert_2^2$, ends up being the same as the vector $\vec{w}^*$ that minimizes mean squared error for the original, uncentered $X$ and $\vec{y}$, just without the intercept term $w_0^*$.

In other words:

$$\text{components $1$ to $d$ of } \left[ (X^TX)^{-1} X^T \vec{y} \right] = (X_c^T X_c)^{-1}X^T \vec{y}_c$$

**We won't prove this fact here, but you're welcome to on your own. For now, take it for granted that centering the data doesn't change any of the coefficients, just the intercept.**
</div>

Just so you believe us, here's an empirical example of this claim.

In [ ]:
X = np.array([[1, 5, 3],
              [1, -1, 2.5],
              [1, 14, -3.14],
              [1, 1998, 15]])

y = np.array([1, 0, 3, 9])

np.linalg.solve(X.T @ X, X.T @ y)

In [ ]:
X_c = X - X.mean(axis=0)
X_c

In [ ]:
X_c = X_c[:, 1:]
X_c

In [ ]:
y_c = y - y.mean()
y_c

In [ ]:
np.linalg.solve(X_c.T @ X_c, X_c.T @ y)

**Note the coefficients of 0.00670059 and -0.39950019 are the same as we saw in the call to `np.linalg.solve(X.T @ X, X.T @ y)`.**

Okay – one last set of definitions. Again, this all has a purpose!

We define the **adjusted** design matrix and observation vector – $X_\text{adj}$ and $\vec{y}_\text{adj}$, respectively – as follows:

$$X_\text{adj} = \begin{bmatrix} X_c \\ \sqrt{n \lambda_\text{ridge}}\cdot I_{d \times d} \end{bmatrix} = \begin{bmatrix}x_1^{(1)} - \mu_1 && x_1^{(2)} - \mu_2 && ... && x_1^{(d)} - \mu_d \\ x_2^{(1)} - \mu_1 && x_2^{(2)} - \mu_2 && ... && x_2^{(d)} - \mu_d \\ \vdots && \vdots && \vdots && \vdots \\ x_n^{(1)} - \mu_1 && x_n^{(2)} - \mu_2 && ... && x_n^{(d)} - \mu_d \\ \sqrt{n \lambda_\text{ridge}} && 0 && ... && 0 \\ 0 && \sqrt{n \lambda_\text{ridge}} && ... && 0 \\ \vdots && \vdots && ... && \vdots \\ 0 && 0 && ... && \sqrt{n \lambda_\text{ridge}} \end{bmatrix}_{\:(n + d) \times d} \qquad \vec{y}_\text{adj} = \begin{bmatrix}y_c \\ \vec 0_{d \times 1} \end{bmatrix} = \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \\ 0 \\ 0 \\ \vdots \\ 0 \end{bmatrix}_{\:(n + d) \times 1}$$

In short:
- To create $X_\text{adj}$, we take $X_c$ and "append" to the bottom a $d \times d$ diagonal matrix, with 0s everywhere except the diagonal, which contains $\sqrt{n \lambda_\text{ridge}}$.
- To create $\vec{y}_\text{adj}$, we take $\vec{y}_c$ and add $d$ new values to the bottom, all of which are 0.

**It turns out that we can use $X_\text{aug}$ and $\vec{y}_\text{aug}$ to show why the ridge regression objective function is minimized by $\left(X_c^TX_c + n\lambda_\text{ridge} I \right)^{-1}X_c^T \vec{y_c}$, which (ignoring the intercept term, which isn't regularized) is the same as $\left(X^TX + n \lambda_\text{ridge} I \right)^{-1} X^T \vec{y}$.**

The high-level gist is that:

$$\text{the $\vec{w}^*$ that minimizes} \frac{1}{n} \lVert \vec{y}_c - X_c \vec{w} \rVert_2^2 + \lambda_\text{ridge} \sum_{j = 1}^d w_j^2$$$$\text{is equal to}$$$$\text{the $\vec{w}^*$ that minimizes} \frac{1}{n} \lVert \vec{y}_\text{adj} - X_\text{adj} \vec{w} \rVert_2^2$$$$\text{both of which are } \left(X_c^TX_c + n\lambda_\text{ridge} I \right)^{-1}X_c^T \vec{y_c}$$

<div class="alert alert-success" markdown="1">

We'll break the derivation down into four steps:
1. Show that $X_\text{adj}^TX_\text{adj} = X_c^TX_c + n \lambda_\text{ridge} I$.
1. Show that $ X_{\text{adj}}^T \vec{y}_\text{adj} = X_c^T \vec{y}_c $.
1. Show that $\frac{1}{n} \lVert \vec{y}_{\text{adj}} - X_{\text{adj}} \vec{w} \rVert_2^2 = \frac{1}{n} \lVert \vec{y}_c - X_c \vec{w} \rVert_2^2 + \lambda_\text{ridge} \sum_{j = 1}^d w_j^2$.
1. Put everything together to demonstrate why $\left(X_c^TX_c + n\lambda_\text{ridge} I \right)^{-1}X_c^T \vec{y_c}$ minimizes the ridge regression objective function.

</div>

We'll work through Step 1 below, but you'll do Steps 2-4 in Questions 2.4-2.6.

Step 1 is to show that:

$$X_\text{adj}^TX_\text{adj} = X_c^TX_c + n \lambda_\text{ridge} I$$

Note that both of these look like the sort of terms we invert, when doing non-regularized linear regression and ridge regression respectively.

Let's examine what the product and shapes are. Here, suppose $\vec{x^{(i)}}$ represents column $i$ of $X_c$, i.e. $\vec{x^{(i)}}$ is already centered.

$$
X^T_\text{adj} X_\text{adj} = \begin{bmatrix} \rule[.5ex]{1.5em}{0.4pt} \ \vec{x^{(1)}} \ \rule[.5ex]{1.5em}{0.4pt} & \sqrt{n\lambda_\text{ridge}}  & 0 & \dots & \dots\\ 
\rule[.5ex]{1.5em}{0.4pt} \ \vec{x^{(2)}} \ \rule[.5ex]{1.5em}{0.4pt} & 0 & \sqrt{n\lambda_\text{ridge}}  & \dots  & \dots\\ 
\dots & \dots & \dots & \dots & \dots \\
\rule[.5ex]{1.5em}{0.4pt} \ \vec{x^{(d)}} \ \rule[.5ex]{1.5em}{0.4pt} & 0 & 0 &  \dots & \sqrt{n\lambda_\text{ridge}}
\end{bmatrix}_{\:d \times (n+d)} 
\begin{bmatrix}  | & | & \dots & | \\ \vec{x^{(1)}} & \vec{x^{(2)}} & \dots & \vec{x^{(d)}} \\ | & | & \dots & | \\
\sqrt{n\lambda_\text{ridge}} & 0 & \dots & \dots \\ 
0 & \sqrt{n\lambda_\text{ridge}} & \dots & \dots \\ 
\dots & \dots & \dots & \sqrt{n\lambda_\text{ridge}} \\
\end{bmatrix}_{\:(n+d) \times d}
$$

What do we know about the product, $X_\text{adj}^T X_\text{adj}$?

- It has shape $d \times d$, just like $X_c^T X_c$.
- Each element in $X_\text{adj}^T X_\text{adj}$ is a **dot product** between a row in $X_\text{adj}^T$ and a column in $X_\text{adj}$. But since the rows of $X_\text{adj}^T$ are just the columns of $X_\text{adj}$, each element in $X_\text{adj}^T X_\text{adj}$ is a **dot product between two columns of $X_\text{adj}$**.
- In general, the number at position $(i, j)$ in $X_\text{adj}^T X_\text{adj}$ is the dot product between column $i$ of $X_\text{adj}$ and column $j$ of $X_\text{adj}$.

There are two cases to consider:

**Case 1**: $i = j$. Here, we're taking a dot product between two identical vectors of shape $(n + d) \times 1$:

$$\begin{bmatrix}  | \\  \vec{x^{(i)}} \\ | \\ 0 \\ \vdots \\ \sqrt{n \lambda_\text{ridge}} \\ 0 \\ \vdots \\ 0 \end{bmatrix} \cdot \begin{bmatrix}  | \\  \vec{x^{(i)}} \\ | \\ 0 \\ \vdots \\ \sqrt{n \lambda_\text{ridge}} \\ 0 \\ \vdots \\ 0 \end{bmatrix}$$

Here, the dot product in the first $n$ components, $\sum_{p = 1}^n x_p^{(i)} \times x_p^{(i)}$, is just the dot product of $\vec{x^{(i)}}$ with $\vec{x^{(i)}}$. Looking at the latter $d$ components, both vectors have the value 0 in $d - 1$ of those components, with a non-zero value of $\sqrt{n \lambda_\text{ridge}}$ **in the same position**. So, the dot product of column $i$ with column $i$ in $X_\text{adj}^T X_\text{adj}$ ends up being:

$$\vec{x^{(i)}} \cdot \vec{x^{(i)}} + n \lambda_\text{ridge}$$

Note that this is equal to the dot product of column $i$ with column $i$ as computed in $X_c^T X_c$, just with the term $\sqrt{n \lambda}$ added.

**Case 2**: $i \neq j$. The difference here is that the $\sqrt{n \lambda_\text{ridge}}$ in the two vectors won't be in the same position. Rather, their dot product will look something like:

$$\begin{bmatrix}  | \\  \vec{x^{(i)}} \\ | \\ 0 \\ \vdots \\ \sqrt{n \lambda_\text{ridge}} \\ 0 \\ \vdots \\ 0 \end{bmatrix} \cdot \begin{bmatrix}  | \\  \vec{x^{(j)}} \\ | \\ 0 \\ \vdots \\ 0 \\ \sqrt{n \lambda_\text{ridge}} \\ \vdots \\ 0 \end{bmatrix}$$

The dot product of column $i$ with column $j$ in $X_\text{adj}^T X_\text{adj}$ ends up being just $\vec{x^{(i)}} \cdot \vec{x^{(j)}}$, i.e, **no different** than in $X_c^T X_c$, since the $\sqrt{n \lambda_\text{ridge}}$ in each column ends up being multiplied by a 0 in the other column.

So, putting this together, the terms of $X_\text{adj}^T X_\text{adj}$ are the same as the terms of $X_c^T X_c$, **the only difference being that** the diagonal terms (where $i = j$, i.e. row number = column number) have $n \lambda_\text{ridge}$ added to them. To represent this, we can take $X_c^T X_c$ and add the identity matrix, $I_{d \times d} = \begin{bmatrix} 1 & 0 & ... & 0 \\ 0 & 1 & ... & 0 \\ \vdots & \vdots & \vdots & \vdots \\ 0 & 0 & ... & 1 \end{bmatrix}$, which is already diagonal with the same constant in every position, multiplied by $n \lambda_\text{ridge}$. 

This means that: $$X_\text{adj}^T X_\text{adj} = X_c^T X_c + n \lambda_\text{ridge} I$$

As needed!

Before proceeding, revisit our outline of what Steps 1-4 of this derivation are so you're clear on what the purpose of this step was, and what comes next.

<!-- BEGIN QUESTION -->

### Question 2.4 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Now, it's your turn to complete Step 2 of the derivation.

Show that:

$$ X_{\text{adj}}^T \vec{y}_\text{adj} = X_c^T \vec{y}_c $$

Some guidance: Make sure you've carefully read our work in Step 1, because you'll need to follow a similar sequence of reasoning; when multiplying $X_\text{adj}^T \vec{y}_\text{adj}$, many of the terms in the resulting matrix are 0—make sure you're clear on which terms those are.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 2.5 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>
Step 3 of the derivation relates the objective functions we seek to minimize in linear and ridge regression. Again, we'll reason about how the adjusted terms $X_{\text{adj}}$ and $\vec{y}_\text{adj}$, in the context of un-regularized linear regression, relate to the ridge regression objective function. Using the work done in Question 2.4 and before, show that:

$$\frac{1}{n} \lVert \vec{y}_{\text{adj}} - X_{\text{adj}} \vec{w} \rVert_2^2 = \frac{1}{n} \lVert \vec{y}_c - X_c \vec{w} \rVert_2^2 + \lambda_\text{ridge} \sum_{j = 1}^d w_j^2$$

Some guidance: 
- One way to proceed is to recall the definition $\lVert \vec v \rVert_2^2 = \vec v \cdot \vec v = \vec v^T \vec v$ and expand from there. Another is to re-write $\lVert \vec v \rVert_2^2 = \sum_{i = 1}^n v_i^2$ and separate sums from there.
- Note that $\sum_{j = 1}^d w_j^2 = \lVert \vec w \rVert_2^2$; we've written it in this summation notation throughout to remain consistent with the notation from class.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 2.6 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Finally, argue why:
   $$
   \vec{w}^* = \left(X_c^T X_c + n \lambda_\text{ridge} I\right)^{-1} X_c^T \vec{y}
   $$

minimizes the ridge regression objective function: $$\frac{1}{n} \lVert \vec{y}_c - X_c \vec{w} \rVert_2^2 + \lambda_\text{ridge} \sum_{j = 1}^d w_j^2$$

<!-- END QUESTION -->

Thanks for bearing with us through this (extremely) long question! We hope you've left it with a deeper understanding of what ridge regression is, and how and why it works.

## Question 3: In This Economy? 🏡

---

In this question, you'll put your understanding of `sklearn` Pipeline objects and cross-validation to practical use as you aim to predict housing prices. The dataset we're using, originally compiled by Professor Dean De Cock at Truman State University **specifically for** teaching regression, contains information about houses sold in Ames, Iowa from 2006 to 2010.

Run the cell below to load in a **subset of the full dataset, which we've designated as your training set**:

In [ ]:
houses_training = pd.read_csv('data/houses-training.csv')
houses_training

There are 82 columns in the dataset! `'SalePrice'` is what we're aiming to predict; everything else _could_ be used as a feature. You'll notice there are some categorical features (some ordinal, some nominal) and some numeric features, and many missing values. Many of the features are self-explanatory, but some are not. Rather than trying to define each feature ourselves, we'll point you to the data description written by the curator of the dataset.

<center><b>Read the data description <a href="https://jse.amstat.org/v19n3/decock/DataDocumentation.txt">here</a>.</center>

Before we build any models, as always, we should explore the data.

In the cell below, draw a histogram depicting the distribution of `'SalePrice'`.

In [ ]:
...

In the cell below, draw a scatter plot of `'SalePrice'` vs. `'Gr Liv Area'` (which represents square footage, not including the basement).

In [ ]:
...

Normally, we'd have you perform a train-test split. However, we've already done this for you, in that `houses_training` is just the training data for this dataset.

In [ ]:
X_train = houses_training.drop(columns=['SalePrice'])
X_train.head()

In [ ]:
y_train = houses_training['SalePrice']
y_train.head()

The test set's features are below. Note that there is no `'SalePrice'` column in this DataFrame.

In [ ]:
X_test = pd.read_csv('data/housing-test-X.csv')
X_test.head()

The test set's actual `'SalePrice'` values (i.e., actual $y$-values) are **intentionally** hidden from you. You won't need them at all in Questions 3.1-3.3. In Question 3.4, you'll have the (optional) opportunity to enter a prediction competition, in which you engineer a model that minimizes testing mean squared error. If you enter, your predictions will be compared against the true `'SalePrice'` values in the test set.

For now, let's just work with `X_train` and `y_train`.

### Question 3.1 [Autograded 💻] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">4 Points</div>

To start, you'll build a `sklearn` Pipeline that does the following to predict `'SalePrice'`:

- Creates a new feature that results from **adding** `'Gr Liv Area'` (non-basement square footage) and `'Total Bsmt SF'` (basement square footage). This is the total square footage of the house.
- Creates degree-2 polynomial features from the total square footage feature defined above.
- One hot encodes `'Neighborhood'`.
- Fits a `LinearRegression` model.

Complete the implementation of the function `create_pipe_sqft_and_neighborhood`, which takes in a DataFrame like `X_train` and a Series like `y_train` and returns a **fit** Pipeline that follows all of the steps above.

Example behavior is given below.

```python
>>> pipe = create_pipe_sqft_and_neighborhood(X_train, y_train)
>>> pipe
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <img src="imgs/first-example-pipe.png" width=400>

```python
>>> pipe.predict(pd.DataFrame([{
    'Gr Liv Area': 2500,
    'Total Bsmt SF': 1500,
    'Neighborhood': 'CollgCr'
}]))
array([312269.07580775])

```


Some guidance:
- All transformations should be done within the Pipeline – you **cannot** preprocess the training data using vanilla `pandas` before creating your Pipeline!
    - So, for instance, to create the total square footage feature, you should create a `FunctionTransformer` that takes in a DataFrame with two columns, adds those two columns, and returns a new DataFrame with a single column that contains the result. This `FunctionTransformer` should be part of a larger Pipeline whose next step is a `OneHotEncoder`.
    - Remember, a `ColumnTransformer` – which you can create easily using `make_column_transformer` – is how you specify which transformations you want applied to which columns. In this particular case, you may want to make a (nested) Pipeline that does the two steps above (namely, the `FunctionTransformer` and `OneHotEncoder`), and then tell the `ColumnTransformer` that you want to use this nested Pipeline on just the two original square footage columns.
    - If you try and preprocess the data using `pandas` before creating your final Pipeline, the example call to `pipe.predict` at the bottom of the cell below won't work.
    - It's okay if the graphical representation of your Pipeline isn't exactly the same as ours.
- Remember to set `include_bias=False` when creating `PolynomialFeatures` so that your model doesn't end up trying to create two intercept terms.
- Remember to use `drop='first'` when using `OneHotEncoder` to avoid multicollinearity, and `handle_unknown='ignore'` so that your Pipeline doesn't error if we try to predict the `'SalePrice'` of a house in a `'Neighborhood'` we've never seen before.

In [ ]:
from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures, OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression

def create_pipe_sqft_and_neighborhood(X_train, y_train):
    ...

# Feel free to change this input to make sure your function works correctly.
# In particular, try changing X_train and y_train to something like
# X_train.head(20) and y_train.head(20)!
pipe = create_pipe_sqft_and_neighborhood(X_train, y_train)
pipe
# Once the above looks right, uncomment the expression below.
# pipe.predict(pd.DataFrame([{
#     'Gr Liv Area': 2500,
#     'Total Bsmt SF': 1500,
#     'Neighborhood': 'CollgCr'
# }]))

In [ ]:
grader.check("q03_01")

### Question 3.2 [Autograded 💻] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">5 Points</div>

Now, let's create a Pipeline that executes all of the steps that `create_pipe_sqft_and_neighborhood` does, but:

1. Instead of fixing the degree of `PolynomialFeatures` to 2, try any possible degree from 1 to 5 (inclusive).
2. Instead of using `LinearRegression`, use `Ridge`, i.e. $L_2$-regularized linear regression. Try regularization penalties $2^{-5}, 2^{-4}, ..., 2^{8}, 2^{9}$, plus $0$. (In class, we called the regularization penalty hyperparameter $\lambda$, but `sklearn` calls it `alpha`.)

The biggest difference here is that you need to use **cross-validation** to choose with polynomial degree and regularization penalty. Use `GridSearchCV` to do this; use the default number of folds. **Remember to tell `GridSearchCV` that you want the hyperparameter combination that yields the lowest mean squared error**; by default, this is not what it does!

Complete the implementation of the function `create_pipe_cross_validated_degree_ridge`, which takes in a DataFrame like `X_train` and a Series like `y_train` and returns a **fit** Pipeline that follows all of the steps above.

Example behavior is given below.

```python
>>> pipe_cv = create_pipe_cross_validated_degree_ridge(X_train, y_train)
>>> pipe_cv
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <img src="imgs/second-example-pipe.png" width=400>

```python
>>> pipe_cv.predict(pd.DataFrame([{
    'Gr Liv Area': 2500,
    'Total Bsmt SF': 1500,
    'Neighborhood': 'CollgCr'
}]))
array([304356.58921189])

```

Some guidance:
- At some point, you'll need to create a grid of hyperparameters to pass to `GridSearchCV`. You'll need to supply this grid as a **dictionary**, mapping hyperparameter names to lists (or ranges) of values. When doing this, the hyperparameter names will be a bit more complicated than seen in lecture – for instance, the `PolynomialFeatures` `degree` hyperparameter is a hyperparameter of a nested Pipeline, which itself is likely part of a `ColumnTransformer`, so the key for the degree will likely look something like `'columntransformer__pipeline__ ...'`.
- If you're getting the wrong output for the example prediction above, verify that you've set the `scoring` argument of `GridSearchCV` correctly.
- `create_pipe_cross_validated_degree_ridge` shouldn't run instantly – it may take ~5 seconds to run. This means that `grader.check("q03_02")` won't run instantly either; it may take ~20 seconds to run.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

def create_pipe_cross_validated_degree_ridge(X_train, y_train):
    ...

# Feel free to change this input to make sure your function works correctly.
pipe_cv = create_pipe_cross_validated_degree_ridge(X_train, y_train)
pipe_cv
# Once the above looks right, uncomment the expression below.
# pipe_cv.predict(pd.DataFrame([{
#     'Gr Liv Area': 2500,
#     'Total Bsmt SF': 1500,
#     'Neighborhood': 'CollgCr'
# }]))

In [ ]:
grader.check("q03_02")

### Question 3.3 [Autograded 💻] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">4 Points</div>

So far, we've only used a small subset of the features in `X_train`. But, there are significantly more!

For our final Pipeline, you're required to use **all features in the dataset**. You'll need to handle numeric and categorical features separately; you can extract all of the numeric columns in a DataFrame using `df.select_dtypes('number')`, for example.

- For **numeric features**:
    - Very few columns have missing values. (For your own exploration, determine which columns these are.) One guess is that these values are missing when the house doesn't have one of those features, e.g. a missing `'Bsmt Half Bath'` must mean that the house doesn't have a basement half-bathroom. **So, use `SimpleImputer` to fill all of the missing numerical values with 0.**
    - Then, one missing values are imputed, standardize all numeric features (and only numeric features!) using a `StandardScaler`.
- For **categorical features**:
    - There are many more missing values. **Use `SimpleImputer` to fill all of the missing values in a column with the _most frequently observed_ value in that column.**
    - Then, one hot encode the resulting categorical columns, making sure to use the same arguments you did earlier to handle multicollinearity and unknown categories (`drop='first'` and `handle_unknown='ignore'`).

After you've created all of your features, fit a `Lasso` object. Use cross-validation to try different regularization penalties from $10^{0}, 10^{1}, ..., 10^{5}$, plus $0$; again, make sure `GridSearchCV` knows that you want the hyperparameter that minimizes mean squared error. Note the different range of hyperparameters as compared to before!

Complete the implementation of the function `create_pipe_all_features_lasso`, which takes in a DataFrame like `X_train` and a Series like `y_train` and returns a **fit** Pipeline that follows all of the steps above.

Example behavior is given below.

```python
>>> pipe_all = create_pipe_all_features_lasso(X_train, y_train)
>>> pipe_all
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <img src="imgs/third-example-pipe.png" width=400>

```python
>>> pipe_all.predict(X_train.head(1))
array([175205.68038145])

```


Some guidance:
- **Pipelines involving `Lasso` take significantly longer to `fit` than Pipelines involving `Ridge`.** This may be due to the fact that the LASSO objective function involves non-differentiable pieces that are harder to optimize. Eventually, when you call `create_pipe_all_features_lasso`, it may take ~1 minute to run.
- To make sure that your transformations are working correctly without having to wait a minute each time you want to test them out, you may want to start by just providing a single regularization penalty hyperparameter for `GridSearchCV` to choose. Once that works without error, switch to providing the range specified above.

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

def create_pipe_all_features_lasso(X_train, y_train):
    ...

# Feel free to change this input to make sure your function works correctly.
pipe_all = create_pipe_all_features_lasso(X_train, y_train)
pipe_all
# Once the above looks right, uncomment the expression below.
# pipe_all.predict(X_train.head(1))

In [ ]:
grader.check("q03_03")

We've now created three pipelines, all of which predict `'SalePrice'` using some combination of features in `X_train`. The Pipeline that uses **all** features has the lowest training mean squared error, by far:

In [ ]:
models = {'pipe (3.1)': pipe, 'pipe_cv (3.2)': pipe_cv, 'pipe_all (3.3)': pipe_all}
for model in models:
    mse = mean_squared_error(y_train, models[model].predict(X_train))
    print(f'Mean squared error for {model}: {mse:.2e}')

But, it's still not clear which of the three Pipelines generalize best to unseen test data. While we used cross-validation to fit `pipe_cv` and `pipe_all`, we didn't directly compare them to one another when doing cross-validation, so it's _possible_ that `pipe_cv` generalizes better than `pipe_all`.

Of course, we **do** have a test set that we could use to assess how well these Pipelines all generalize, but we can't give you access to it just yet.

One last thing before Question 3.4: recall, LASSO (which we used in `pipe_all`) encourages **sparsity**, meaning that we should expect the coefficients of many features to end up being 0. We can see exactly which features had a coefficient of 0 in `pipe_all` here:

In [ ]:
feature_names = pipe_all.best_estimator_[:-1].get_feature_names_out()
coefficients = pipe_all.best_estimator_[-1].coef_

In [ ]:
coefs = pd.Series(coefficients, index=feature_names)
coefs[coefs == 0]

So, LASSO was implicitly telling us **not** to use those features, if we care about building a model that generalizes well to unseen data! This feature selection process might be useful to you should you choose to complete Question 3.4.

### Optional: Question 3.4 [Autograded 💻] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">Extra Credit</div>

**This part of the question is OPTIONAL, and just counts for extra credit!**

In Questions 3.1 through 3.3, we specified exactly how you should create your Pipelines. But now, it's your job to choose features and transformations that make good, generalizable predictions.

If you decide to complete Question 3.4, here's the process.
1. Below, complete the implementation of the function `create_pipe_custom`, which takes in a DataFrame like `X_train` and a Series like `y_train` and returns a **fit** Pipeline, created however you'd like.
2. Then, run the cell below so that `pipe_custom` is assigned to a a fit Pipeline.
3. Run the three cells below this one, that starts with `# EXPORT CELL!`, to create a CSV of your Pipeline's predictions on the hidden test set.
4. Upload the CSV created (it should be named something like `'predictions-2024-11.....csv'`) to the **Homework 10, Question 3.4 Leaderboard (Optional!)** autograder on Gradescope.
5. Ignore the "score" that appears on Gradescope, since everyone receives a score of 0 – all that matters is your ranking on the leaderboard, linked [**here**](https://www.gradescope.com/courses/823979/assignments/5342865/leaderboard). **The rankings are computed using your mean squared error on the unseen testing set; the lower the MSE, the higher your ranking.**
6. The top 5 **good-faith** submissions on the leaderboard will receive extra credit on Homework 10 as follows:
    - The top submission – that is, the one with the lowest MSE – will earn 20 points of extra credit on Homework 10. Since the homework is out of 80 points, this equates to **25% of extra credit**.
    - The 2nd-best submission will earn 16 extra points on Homework 10 (20% of extra credit).
    - The 3rd-best submission will earn 12 extra points on Homework 10 (15% of extra credit).
    - The 4th-best submission will earn 8 extra points on Homework 10 (10% of extra credit).
    - The 5th-best submission will earn 4 extra points on Homework 10 (5% of extra credit).
     
Some guidance:
- You can use _any_ regression class **in `sklearn`** to make your predictions. By **good-faith submission**, we mean a submission that doesn't somehow determine the true $y$-values in the test set and hardcodes them, and a submission that is unique, i.e. not copied from someone else (that would, of course, be an honor code violation). Before assigning extra credit, we will manually inspect your submitted notebook (which you need to submit anyways for the rest of the homework to be graded), and any submissions that don't use an `sklearn` Pipeline will not receive the extra credit.
- Don't just guess arbitrarily which features might be useful and how to engineer them. **Do some exploratory data analysis!** Look at the relationships between various features and `'SalePrice'`. You might discover various new features you want to engineer.
- For reference, you'll find a submission titled **Submitted by Suraj: Baseline Model from Question 3.3** on the leaderboard. This shows you the test MSE of the Pipeline that `create_pipe_all_features_lasso` achieves. Your model's MSE should be lower than this!
- Have fun with it, and use what you've learned in this question to improve your models in the Portfolio Homework!

In [ ]:
def create_pipe_custom(X_train, y_train):
    ...

# Make sure this has been run before you try and run the export cell below!
pipe_custom = create_pipe_custom(X_train, y_train)
pipe_custom

Once you've implemented `create_pipe_custom` and defined `pipe_custom` at the bottom of the cell above, run the cell below to generate your CSV of test set predictions. Upload this CSV to the Gradescope assignment titled **Homework 10, Question 3.4 Leaderboard (Optional!)**.

In [ ]:
# EXPORT CELL!

import datetime
current_time = str(datetime.datetime.now())[:19]

y_pred = pipe_custom.predict(X_test)
y_pred_df = pd.DataFrame().assign(predictions=y_pred)
y_pred_df.to_csv(f'test-predictions-{current_time}.csv', index=False)
print(f'Saved test-predictions-{current_time}.csv; upload this to Gradescope.') 

## Question 4: Meet the Jensens 🧑‍🧑‍🧒‍🧒

---

As we've seen several times, the variance of a dataset $x_1, x_2, \ldots, x_n$ is defined:

$$\sigma_x^2 = \frac{1}{n} \sum_{i = 1}^n (x_i - \bar{x})^2$$

where $\bar{x} = \text{Mean}(x_1, x_2, \ldots, x_n)$. By expanding the summation, we find that:

$$\sigma_x^2 = \frac{1}{n} \sum_{i = 1}^n x_i^2 - \bar{x}^2$$

Another way of expressing this equation is:

$$\sigma_x^2 = \text{Mean}(x_1^2, x_2^2, \ldots, x_n^2) - \left( \text{Mean}(x_1, x_2, \ldots, x_n) \right)^2$$

Since $\sigma_x^2 \geq 0$, this implies that:

$$\begin{align*} \text{Mean}(x_1^2, x_2^2, \ldots, x_n^2) - \left( \text{Mean}(x_1, x_2, \ldots, x_n) \right)^2 &\geq 0 \\ \implies \text{Mean}(x_1^2, x_2^2, \ldots, x_n^2) &\geq \left( \text{Mean}(x_1, x_2, \ldots, x_n) \right)^2\end{align*}$$

The inequality on the last line can be expressed more generally as follows (where $g(x) = x^2$):

$$\boxed{\text{Mean}(g(x_1), g(x_2), \ldots, g(x_n)) \geq g\left( \text{Mean}(x_1, x_2, \ldots, x_n) \right)}$$

The inequality above is known as Jensen's inequality, and is true for **all convex functions** $g$. Let's see how we can use Jensen's inequality to prove something useful!

<!-- BEGIN QUESTION -->

### Question 4.1 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>
Prove that the function $g(x) = - \log x$ is convex. 

Some guidance: Use the second derivative test; it'd be difficult to prove this with the formal definition of convexity.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 4.2 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">4 Points</div>

Using Jensen's inequality and $g(x) = - \log x$, prove that, for any dataset of positive numbers $x_1, x_2, \ldots, x_n$:

$$\frac{x_1 + x_2 + \ldots + x_n}{n} \geq \left( x_1 \cdot x_2 \cdot \ldots \cdot x_n \right)^{\frac{1}{n}}$$

The quantity on the left is the familiar arithmetic mean (AM), while the quantity on the right is known as the geometric mean (GM) of $x_1, x_2, \ldots, x_n$. The entire inequality above is known as the "AM-GM inequality."

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 4.3 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">4 Points</div>

Using Jensen's inequality and a convex function $g$, prove that the arithmetic mean is greater than or equal to the harmonic mean for any dataset of positive numbers $x_1, x_2, \ldots, x_n$, i.e.:$$\frac{x_1 + x_2 + \ldots + x_n}{n} \geq \frac{n}{\displaystyle \frac{1}{x_1} + \frac{1}{x_2} + \ldots + \frac{1}{x_n}}$$

Some guidance:
- You must prove your choice of function $g$ is convex!
- You can use a function that is only convex on an interval, as long as the only inputs you pass into the function come from that interval.

<!-- END QUESTION -->

## Question 5: Gradient Descent Gone Wrong 😱

---

In this question, we'll familiarize ourselves with how gradient descent works. As mentioned in class, while gradient descent *can* be used to minimize arbitrary differentiable functions, it's most commonly used to find optimal model parameters in the context of empirical risk minimization.

Let's suppose we want to fit a constant model, $H(x) = h$, using degree-3 loss:

$$L_\text{3} (y_i, h) = (y_i - h)^3$$

We're given $y_1 = 1, y_2 = 1, y_3 = 7$. Recall, the goal is to find $h^*$, the best constant prediction for this loss function and dataset.

<!-- BEGIN QUESTION -->

### Question 5.1 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

Find $R_3(h)$ and $\frac{dR_3}{dh}(h)$ for this dataset. Both should only involve the variable $h$; everything else should be constants. (The term "$y_i$" should not appear in either formula.)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 5.2 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">4 Points</div>

Given an initial guess $h^{(0)} = 1$ and a learning rate of $\alpha = \frac{1}{9}$, perform two iterations of gradient descent. What are $h^{(1)}$ and $h^{(2)}$?

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 5.3 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Is it possible to find an initial guess, $h^{(0)}$, and learning rate, $\alpha$, for which gradient descent will minimize $R_3(h)$? Why or why not? If we did want to minimize degree-3 loss, how must we change the loss function to ensure it works?

<!-- END QUESTION -->

<br>

Now that we've gotten a feel for how to use gradient descent to minimize a function of a single variable, we'll use it to minimize a function of multiple variables. Let's suppose we want to fit a simple linear regression model, $H(x) = w_0 + w_1 x$, using **squared loss**. We're searching for the optimal parameters $w_0^*$ and $w_1^*$, which we can write in vector form as $\vec{w}^* = \begin{bmatrix} w_0^* \\ w_1^* \end{bmatrix}$.

We're given the following dataset of $(x, y)$ pairs: $\{ (1, 5), (2, 7) \}$.

<!-- BEGIN QUESTION -->

### Question 5.4 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

Find the average loss $R_\text{sq}(\vec{w}) = R_\text{sq}(w_0, w_1)$ and the gradient vector: $$\displaystyle \nabla R_\text{sq}(\vec{w}) = \begin{bmatrix}\frac{\partial R_\text{sq}}{\partial w_0} \\ \frac{\partial R_\text{sq}}{\partial w_1}\end{bmatrix}$$

Both $R_\text{sq}(\vec{w})$ and $\displaystyle \nabla R_\text{sq}(\vec{w})$ should only involve the variables $w_0$ and $w_1$; everything else should be constants.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 5.5 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

Given an initial guess $\vec{w}^{(0)} = \begin{bmatrix} 0 \\ 2 \end{bmatrix}$ and a learning rate of $\alpha = \frac{1}{3}$, perform one iteration of gradient descent according to the update rule:
$$\vec{w}^{(t+1)} = \vec{w}^{(t)} - \alpha \nabla R(\vec{w}^{(t)}) $$
What are the components of $\vec{w}^{(1)}$?



<!-- END QUESTION -->

That's it for gradient descent in this homework. We promise, you'll look at cooler, more practical examples in Homework 11!

## Question 6: Wall Street Logistics 💰

---

In this question, you'll strengthen your theoretical understanding of logistic regression, a classification technique covered in detail in [Lecture 24](https://practicaldsc.org/resources/lectures/lec24/lec24-filled.html).

WolverineExpress, a credit card company, has tasked you with building a classification model to predict whether or not customers will fail to pay their next credit card payment. You've decided to use logistic regression to build the classifier.

For context, the first few rows of the dataset are given below, though you don't need any specific numbers from this DataFrame preview in the problem.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>education</th>
      <th>marriage</th>
      <th>age</th>
      <th>failed payment</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>1</td>
      <td>40</td>
      <td>1</td>
    </tr>
    <tr>
      <td>1</td>
      <td>2</td>
      <td>23</td>
      <td>0</td>
    </tr>
    <tr>
      <td>2</td>
      <td>1</td>
      <td>36</td>
      <td>0</td>
    </tr>
    <tr>
      <td>3</td>
      <td>1</td>
      <td>54</td>
      <td>0</td>
    </tr>
    <tr>
      <td>1</td>
      <td>1</td>
      <td>35</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

The columns of the dataset are as follows:
- `'education'`: 1 - graduate school; 2 - university; 3 - high school; 4 - other.
- `'marriage'`: 1 - married; 2 - single; 3 - other.
- `'age'`: customer age in years.

Our target variable, `'failed payment'`, can have values of 0 (makes their next payment) or 1 (fails to make their next payment).

You use the logistic regression model:

$$P(y = 1 | \vec{x}) = \sigma(\vec w \cdot \text{Aug}( \vec x ) ) = \sigma(w_0 + w_1 \cdot \text{education} + w_2 \cdot \text{marriage} + w_3 \cdot \text{age})$$

Assume that the following value of $ \vec{w}^*$ minimizes mean cross-entropy loss (without regularization) for this dataset: 
$$\vec{w}^* = \begin{bmatrix}-\frac{5}{4} \\ \frac{1}{6} \\ -\frac{1}{10} \\ \frac{1}{500}\end{bmatrix}$$ 

Here, $-\frac{5}{4}$ is the intercept term, $\frac{1}{6}$ corresponds to `'education'`, $-\frac{1}{10}$ corresponds to `'marriage'` status, and $\frac{1}{500}$ corresponds to `'age'`.

<!-- BEGIN QUESTION -->

### Question 6.1 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>
Consider a customer who is 50 years old, married, and only has a high school education.  Compute the chance that they fail to pay their next credit card payment.  Give your answer as a probability in terms of $\sigma$.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 6.2 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>
This specific customer fortunately made their next payment on time!  Compute the cross-entropy loss of the prediction in Question 6.1.  Leave your answers in terms of $\sigma$ and $\log$.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 6.3 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">1 Point</div>
How does a one year increase in `'age'` impact the **log-odds** of making a failed payment? Give a precise, numerical answer, not just "it increases" or "it decreases."

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 6.4 [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>
Let's consider all customers who are married and whose highest level of education is high school. What is the **minimum age** of such a customer, such that they more likely to fail their next payment than make their next payment, under our logistic regression model?

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 6.5  [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

Suppose you choose a threshold $T = 0.8$. The decision boundary of the resulting classifier is of the form

$$A \cdot \text{education} + B \cdot \text{marriage} + C \cdot \text{age} + D = 0$$

What are the values of $A$, $B$, $C$, and $D$? Your answers may contain a $\log$, but should not contain $\sigma$. (If your answers do contain a $\log$, don't try and simplify them to numbers.)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 6.6  [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Suppose with the above threshold you achieve a training accuracy of 100%. Can you conclude your training data was linearly separable in the feature space? Answer yes or no, and explain in one sentence.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 6.7  [Written ✏️] <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>
For whatever reason, we decide to multiply the `'education'` feature by 4 and `'age'` feature by 0.5 in our data. What is the new optimal parameter vector, $\vec{w}^*_\text{new}$ that minimizes mean cross-entropy loss (again, without regularization)? 

If you don't believe it's possible to tell, say so. For your convenience, the value of $\vec{w}^*$ that minimized un-regularized mean cross-entropy loss on the original data was $\vec{w}^* = \begin{bmatrix}-\frac{5}{4} \\ \frac{1}{6} \\ -\frac{1}{10} \\ \frac{1}{500}\end{bmatrix}$; here, $-\frac{5}{4}$ is the intercept term, $\frac{1}{6}$ corresponds to `'education'`, $-\frac{1}{10}$ corresponds to `'marriage'` status, and $\frac{1}{500}$ corresponds to `'age'`.

Note: This part is independent of the previous parts, i.e. do not assume that we achieved a training accuracy of 100\%.

<!-- END QUESTION -->

## Finish Line 🏁

Congratulations! You're ready to submit Homework 10. **Remember, you need to submit Homework 10 twice (or three times, if you're participating in the optional competition for Question 3.4)**:

### To submit the manually graded problems (Questions 2, 4, 5, and 6; marked [Written ✏️])

- Make sure your answers **are not** in this notebook, but rather in a separate PDF.
    - You can create this PDF either digitally, using your tablet or using [Overleaf + LaTeX](https://overleaf.com) (or some other sort of digital document), or by writing your answers on a piece of paper and scanning them in.
- Submit this separate PDF to the **Homework 10 (Questions 2, 4, 5, and 6; written problems)** assignment on Gradescope, and **make sure to correctly select the pages associated with each question**!

### To submit the autograded problems (Questions 1 and 3; marked [Autograded 💻])

1. Select `Kernel -> Restart & Run All` to ensure that you have executed all cells, including the test cells.
2. Read through the notebook to make sure everything is fine and all tests passed.
3. Run the cell below to run all tests, and make sure that they all pass.
4. Download your notebook using `File -> Download as -> Notebook (.ipynb)`, then upload your notebook to Gradescope under **"Homework 10 (Questions 1 and 3; autograded problems)"**. Make sure your notebook is still named `hw10.ipynb` and the name has not been changed.
5. Stick around while the Gradescope autograder grades your work.
6. Check that you have a confirmation email from Gradescope and save it as proof of your submission.

Your Homework 10 submission time will be the **later** of your two individual submissions.

### To submit to the optional prediction competition (Question 3.4)

See the details in Question 3.4.